
# Задание 3.1 - Сверточные нейронные сети (Convolutional Neural Networks)

Это последнее задание на numpy, вы до него дожили! Остался последний марш-бросок, дальше только PyTorch.  
В этом задании вы реализуете свою собственную сверточную нейронную сеть.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [3]:
from dataset import load_svhn, random_split_train_val
from gradient_check import check_layer_gradient, check_layer_param_gradient, check_model_gradient
from layers import FullyConnectedLayer, ReLULayer, ConvolutionalLayer, MaxPoolingLayer, Flattener
from model import ConvNet
from trainer import Trainer, Dataset
from optim import SGD, MomentumSGD
from metrics import multiclass_accuracy

### Загружаем данные

На этот раз мы не будем их преобразовывать в один вектор, а оставим размерности `(num_samples, 32, 32, 3)`.

In [4]:
def prepare_for_neural_network(train_X, test_X):    
    train_X = train_X.astype(float) / 255.0
    test_X = test_X.astype(float) / 255.0
    
    # Subtract mean
    mean_image = np.mean(train_X, axis = 0)
    train_X -= mean_image
    test_X -= mean_image
    
    return train_X, test_X
    
train_X, train_y, test_X, test_y = load_svhn("data", max_train=10000, max_test=1000)    
train_X, test_X = prepare_for_neural_network(train_X, test_X)
# Split train into train and val
train_X, train_y, val_X, val_y = random_split_train_val(train_X, train_y, num_val = 1000)

### Реализуем новые слои!
Сначала основной новый слой - сверточный (Convolutional layer). Для начала мы реализуем его для только одного канала, а потом для нескольких.

Сверточный слой выполняет операцию свертки (convolution) с весами для каждого канала, а потом складывает результаты. Возможно, поможет пересмотреть Лекцию 6 или внимательно прочитать http://cs231n.github.io/convolutional-networks/

Один из подходов к реализации сверточного слоя основан на том, что для конкретного "пикселя" выхода применение сверточного слоя эквивалентно обычному полносвязному.

![Getting Started](conv_img.jpg)

Рассмотрим один такой "пиксель":

Он получает на вход  
регион входа I размера `(batch_size, filter_size, filter_size, input_channels)`,

применяет к нему веса W `(filter_size, filter_size, input_channels, output_channels` и выдает `(batch_size, output_channels)`.

Если:  
* вход преобразовать в I' `(batch_size, filter_size*filter_size*input_channels)`,  
* веса в W' `(filter_size*filter_size*input_channels, output_channels)`,

то выход "пикселе" будет эквивалентен полносвязному слою со входом I' и весами W'.  
Осталось выполнить его в цикле для каждого пикселя :)

In [5]:
# TODO: Implement ConvolutionaLayer that supports only 1 output and input channel

# Note: now you're working with images, so X is 4-dimensional tensor of
# (batch_size, height, width, channels)

np.random.seed(0)
X = np.array([
              [
               [[1.0], [2.0]],
               [[0.0], [-1.0]]
              ]
              ,
              [
               [[0.0], [1.0]],
               [[-2.0], [-1.0]]
              ]
             ])

# Batch of 2 images of dimensions 2x2 with a single channel
print("Shape of X:",X.shape)

layer = ConvolutionalLayer(in_channels=1, out_channels=1, filter_size=2, padding=0)
print("Shape of W", layer.W.value.shape)
layer.W.value = np.zeros_like(layer.W.value)
layer.W.value[0, 0, 0, 0] = 1.0
layer.B.value = np.ones_like(layer.B.value)
result = layer.forward(X)

assert result.shape == (2, 1, 1, 1)
assert np.all(result == X[:, :1, :1, :1] +1), "result: %s, X: %s" % (result, X[:, :1, :1, :1])


# Now let's implement multiple output channels
layer = ConvolutionalLayer(in_channels=1, out_channels=2, filter_size=2, padding=0)
result = layer.forward(X)
assert result.shape == (2, 1, 1, 2)


# And now multple input channels!
X = np.array([
              [
               [[1.0, 0.0], [2.0, 1.0]],
               [[0.0, -1.0], [-1.0, -2.0]]
              ]
              ,
              [
               [[0.0, 1.0], [1.0, -1.0]],
               [[-2.0, 2.0], [-1.0, 0.0]]
              ]
             ])

print("Shape of X:", X.shape)
layer = ConvolutionalLayer(in_channels=2, out_channels=2, filter_size=2, padding=0)
result = layer.forward(X)
assert result.shape == (2, 1, 1, 2)

Shape of X: (2, 2, 2, 1)
Shape of W (2, 2, 1, 1)
Shape of X: (2, 2, 2, 2)


### А теперь имплементируем обратный проход

Возможно, это самое сложное место в курсе. Дальше будет лучше.

Раз выполнение сверточного слоя эквивалентно полносвязному слою для каждого "пикселя" выхода, то общий обратный проход эквивалентен обратному проходу каждого из таких "слоев".  
Градиенты от каждого из этих "слоев" в каждом пикселе надо сложить в соответствующие пиксели градиента по входу, а градиенты весов сложить все вместе.

In [38]:
# First test - check the shape is right
layer = ConvolutionalLayer(in_channels=2, out_channels=2, filter_size=2, padding=0)
result = layer.forward(X)
d_input = layer.backward(np.ones_like(result))
assert d_input.shape == X.shape

# # Actually test the backward pass
# # As usual, you'll need to copy gradient check code from the previous assignment
layer = ConvolutionalLayer(in_channels=2, out_channels=2, filter_size=2, padding=0)
assert check_layer_gradient(layer, X)

layer = ConvolutionalLayer(in_channels=2, out_channels=2, filter_size=2, padding=0)
assert check_layer_param_gradient(layer, X, 'W')
layer = ConvolutionalLayer(in_channels=2, out_channels=2, filter_size=2, padding=0)
assert check_layer_param_gradient(layer, X, 'B')

X shape  (2, 2, 2, 2)
d out shape  (2, 1, 1, 2)
d input shape  (2, 2, 2, 2)
resh d out shape  (2, 2)
fragment x shape  (2, 2, 2, 2)
resh X shape  (2, 8)
resh_W.T shape  (2, 8)
d_out[:, y, x, :] shape  (2, 2)
resh_d_input shape  (2, 8)
resh_d_input shape  (2, 2, 2, 2)
CHECK GRADIENT
x is 
 [[[[ 1.  0.]
   [ 2.  1.]]

  [[ 0. -1.]
   [-1. -2.]]]


 [[[ 0.  1.]
   [ 1. -1.]]

  [[-2.  2.]
   [-1.  0.]]]]
X shape  (2, 2, 2, 2)
d out shape  (2, 1, 1, 2)
d input shape  (2, 2, 2, 2)
resh d out shape  (2, 2)
fragment x shape  (2, 2, 2, 2)
resh X shape  (2, 8)
resh_W.T shape  (2, 8)
d_out[:, y, x, :] shape  (2, 2)
resh_d_input shape  (2, 8)
resh_d_input shape  (2, 2, 2, 2)
analytic grad is 
 [[[[ 0.00352054  1.47264106]
   [-0.53732008  0.63296179]]

  [[ 0.59903906  1.20815337]
   [ 0.16412459  0.29955608]]]


 [[[ 0.07789974  3.12069365]
   [-0.93836466  1.65422806]]

  [[ 1.49227624  2.94405433]
   [ 0.4249922   0.43891065]]]]
X shape  (2, 2, 2, 2)
d out shape  (2, 1, 1, 2)
d input shape  (2


Осталось реализовать дополнение нулями (padding).  
Достаточно дополнить входной тензор нулями по сторонам. Не забудьте учесть это при обратном проходе!

In [39]:
X = np.array([
              [
               [[1.0, 0.0], [2.0, 1.0]],
               [[0.0, -1.0], [-1.0, -2.0]]
              ]
              ,
              [
               [[0.0, 1.0], [1.0, -1.0]],
               [[-2.0, 2.0], [-1.0, 0.0]]
              ]
             ])
print("X shape ", X.shape)            
layer = ConvolutionalLayer(in_channels=2, out_channels=2, filter_size=3, padding=1)
result = layer.forward(X)
print("result shape is ", result.shape)
# Note this kind of layer produces the same dimensions as input
assert result.shape == X.shape,"Result shape: %s - Expected shape %s" % (result.shape, X.shape)
d_input = layer.backward(np.ones_like(result))
assert d_input.shape == X.shape
# layer = ConvolutionalLayer(in_channels=2, out_channels=2, filter_size=3, padding=1)
# assert check_layer_gradient(layer, X)

X shape  (2, 2, 2, 2)
result shape is  (2, 2, 2, 2)
X shape  (2, 4, 4, 2)
d out shape  (2, 2, 2, 2)
d input shape  (2, 4, 4, 2)
resh d out shape  (2, 8)
fragment x shape  (2, 3, 3, 2)
resh X shape  (2, 18)
resh_W.T shape  (2, 18)
d_out[:, y, x, :] shape  (2, 2)
resh_d_input shape  (2, 18)
resh_d_input shape  (2, 3, 3, 2)
fragment x shape  (2, 3, 3, 2)
resh X shape  (2, 18)
resh_W.T shape  (2, 18)
d_out[:, y, x, :] shape  (2, 2)
resh_d_input shape  (2, 18)
resh_d_input shape  (2, 3, 3, 2)
fragment x shape  (2, 3, 3, 2)
resh X shape  (2, 18)
resh_W.T shape  (2, 18)
d_out[:, y, x, :] shape  (2, 2)
resh_d_input shape  (2, 18)
resh_d_input shape  (2, 3, 3, 2)
fragment x shape  (2, 3, 3, 2)
resh X shape  (2, 18)
resh_W.T shape  (2, 18)
d_out[:, y, x, :] shape  (2, 2)
resh_d_input shape  (2, 18)
resh_d_input shape  (2, 3, 3, 2)
d_input shape  (2, 2, 2, 2)


In [59]:
X = np.array([
              [
               [[1.0, 1, 0.0], [2.0, 1, 1.0], [9.0, 1, 1.0]],
               [[0.0, 1,  -1.0], [-1.0, 1,  -2.0],[2.0, 1, 1.0]], 
               [[0.0, 1,  -1.0], [-5.0, 1,  -2.0],[2.0, 9, 1.0]],
               [[0.0, 1,  -1.0], [-1.0, 1,  -2.0],[2.0, 1, 1.0]]
              ]
              ,
              [
               [[0.0, 1, 1.0], [1.0, 1,  -1.0], [2.0, 1, 1.0]],
               [[-2.0, 1,  2.0], [-1.0, 1, 0.0], [2.0, 1, 1.0]],
               [[-2.0, 1,  2.0], [-1.0, 1, 0.0], [2.0, 1, 1.0]],
               [[0.0, 1,  -1.0], [-1.0, 1,  -2.0],[2.0, 1, 1.0]]
              ]
             ])

# A = np.array([[[3,3], [2,2]], [[3,3,], [1,1]]])
x_off= 1
y_off = 1
Y = X[:, x_off:2+x_off,  y_off:2 + y_off, :]
print("X shape is ", X.shape)
# print("Y shape is ", Y.shape)
# print(X)
# print(X.ndim)
# print("Y is\n", Y)
# print("A\n", A)
print("transformed X shape is ", X.T.shape)

b = [1000, 100, 30]
print(X + b)


X shape is  (2, 4, 3, 3)
transformed X shape is  (3, 3, 4, 2)
[[[[1001.  101.   30.]
   [1002.  101.   31.]
   [1009.  101.   31.]]

  [[1000.  101.   29.]
   [ 999.  101.   28.]
   [1002.  101.   31.]]

  [[1000.  101.   29.]
   [ 995.  101.   28.]
   [1002.  109.   31.]]

  [[1000.  101.   29.]
   [ 999.  101.   28.]
   [1002.  101.   31.]]]


 [[[1000.  101.   31.]
   [1001.  101.   29.]
   [1002.  101.   31.]]

  [[ 998.  101.   32.]
   [ 999.  101.   30.]
   [1002.  101.   31.]]

  [[ 998.  101.   32.]
   [ 999.  101.   30.]
   [1002.  101.   31.]]

  [[1000.  101.   29.]
   [ 999.  101.   28.]
   [1002.  101.   31.]]]]
